In [683]:
%config Completer.use_jedi = False

In [684]:
import pandas as pd
import glob
import xarray as xr
from datetime import datetime

In [685]:
# Caminhos dos diretórios raiz
# general_path = 'C:/Users/cnalm/OneDrive/Hidroweb'
# general_path = 'D:/Dados_Nuvem/OneDrive/Hidroweb'
general_path = r'C:\Users\linde\OneDrive\Hidroweb'
idw_path = general_path + r'\Consolidated Files\NetCDF\IDW'
cn_path_station = r'C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NormaisClimatologicas\Estacoes'
cn_path_prec = r'C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NormaisClimatologicas\Prec'

In [686]:
all_files = glob.glob(idw_path + "\*.nc")
total_length = len(all_files)
print(total_length)
all_files[10957]

21915


'C:\\Users\\linde\\OneDrive\\Hidroweb\\Consolidated Files\\NetCDF\\IDW\\precipitation_idw_1991-01-01.nc'

In [687]:
def convert_degrees(string):
    parts = string.split('°')
    degrees = float(parts[0])
    minutes = float(parts[1].strip("'").split('\'')[0])
    direction = parts[1].strip("'").split('\'')[1]
    
    decimal_degrees = degrees + minutes / 60
    
    if direction == 'S' or direction == 'W':
        decimal_degrees *= -1
    
    return decimal_degrees

In [688]:
df_cn_station_1961_1990 = pd.read_excel(cn_path_station + "/NC_ESTACOES_1961_1990.xlsx")
df_cn_station_1961_1990 = df_cn_station_1961_1990[["Código",	"Nome",	"UF",	"Latitude",	"Longitude",	"Altitude (m)"]]
df_cn_station_1961_1990['Latitude_Decimal'] = df_cn_station_1961_1990['Latitude'].apply(convert_degrees)
df_cn_station_1961_1990['Longitude_Decimal'] = df_cn_station_1961_1990['Longitude'].apply(convert_degrees)
df_cn_station_1961_1990

,Código,Nome,UF,Latitude,Longitude,Altitude (m),Latitude_Decimal,Longitude_Decimal
0,83010,Brasiléia,AC,11°01'S,68°44'W,260.00,-11.016667,-68.733333
1,82704,Cruzeiro do Sul,AC,07°38'S,72°40'W,170.00,-7.633333,-72.666667
2,82915,Rio Branco,AC,09°58'S,67°48'W,160.00,-9.966667,-67.800000
3,82807,Tarauacá,AC,08°10'S,70°46'W,190.00,-8.166667,-70.766667
4,83098,Coruripe,AL,10°09'S,36°10'W,56.13,-10.150000,-36.166667
...,...,...,...,...,...,...,...,...
389,83231,Paranã,TO,12°33'S,47°50'W,275.00,-12.550000,-47.833333
390,82863,Pedro Afonso,TO,08°58'S,48°11'W,187.00,-8.966667,-48.183333
391,83228,Peixe,TO,12°01'S,48°21'W,242.49,-12.016667,-48.350000
392,83064,Porto Nacional,TO,10°43'S,48°25'W,239.20,-10.716667,-48.416667


In [689]:
def pr_value_calc(lat, lon, df):
    df_temp = df.sel(Longitude = lon, Latitude = lat, method = 'nearest')
    return df_temp.variables['Precipitation'].to_numpy()

In [690]:
count = 0
total_len = 0
df_result = []
for filename in all_files[:10957]:
    date_str = filename[-13:-3]
    ref_date = datetime.strptime(date_str, "%Y-%m-%d")
    print(count, filename, ref_date)
    count = count + 1
    df_temp = df_cn_station_1961_1990.copy(deep=True)
    lat_list = df_temp['Latitude_Decimal'].values.tolist()
    lon_list = df_temp['Longitude_Decimal'].values.tolist()
    coord_list = []
    for i in range(len(lat_list)):
        # print(lat_list[i])
        coord_list.append([lat_list[i], lon_list[i]])
    ds = xr.open_dataset(filename)
    pr_list = []
    for coord in coord_list:
        pr_value = pr_value_calc(coord[0], coord[1], ds)
        pr_list.append(pr_value[0])
        # print(pr_value)
    # print(pr_list)
    df_temp['Value_Interpolated'] = pr_list
    df_temp['Date'] = ref_date
    # print(df_temp.head(5))
    if len(df_result) == 0:
        df_result = df_temp.copy(deep=True)
    else:
        df_result = pd.concat([df_result, df_temp], ignore_index= True)
    total_len = total_len + len(df_temp)
df_result

0 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-01.nc 1961-01-01 00:00:00
1 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-02.nc 1961-01-02 00:00:00
2 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-03.nc 1961-01-03 00:00:00
3 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-04.nc 1961-01-04 00:00:00
4 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-05.nc 1961-01-05 00:00:00
5 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-06.nc 1961-01-06 00:00:00
6 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-07.nc 1961-01-07 00:00:00
7 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1961-01-08.nc 1961-01-08 00:00:00
8 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW

,Código,Nome,UF,Latitude,Longitude,Altitude (m),Latitude_Decimal,Longitude_Decimal,Value_Interpolated,Date
0,83010,Brasiléia,AC,11°01'S,68°44'W,260.00,-11.016667,-68.733333,4.278750,1961-01-01
1,82704,Cruzeiro do Sul,AC,07°38'S,72°40'W,170.00,-7.633333,-72.666667,0.028541,1961-01-01
2,82915,Rio Branco,AC,09°58'S,67°48'W,160.00,-9.966667,-67.800000,2.152689,1961-01-01
3,82807,Tarauacá,AC,08°10'S,70°46'W,190.00,-8.166667,-70.766667,3.906242,1961-01-01
4,83098,Coruripe,AL,10°09'S,36°10'W,56.13,-10.150000,-36.166667,0.000000,1961-01-01
...,...,...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,12°33'S,47°50'W,275.00,-12.550000,-47.833333,0.773416,1990-12-31
4317054,82863,Pedro Afonso,TO,08°58'S,48°11'W,187.00,-8.966667,-48.183333,0.299561,1990-12-31
4317055,83228,Peixe,TO,12°01'S,48°21'W,242.49,-12.016667,-48.350000,5.480674,1990-12-31
4317056,83064,Porto Nacional,TO,10°43'S,48°25'W,239.20,-10.716667,-48.416667,8.465086,1990-12-31


In [699]:
df_check = df_result[df_result['Código'] == 83948]
df_check

,Código,Nome,UF,Latitude,Longitude,Altitude (m),Latitude_Decimal,Longitude_Decimal,Value_Interpolated,Date,Year,Month
331,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.193623,1961-01-01,1961,1
725,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.000000,1961-01-02,1961,1
1119,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,2.255571,1961-01-03,1961,1
1513,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,7.804980,1961-01-04,1961,1
1907,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.022960,1961-01-05,1961,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4315419,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.000000,1990-12-27,1990,12
4315813,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.375475,1990-12-28,1990,12
4316207,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,2.129621,1990-12-29,1990,12
4316601,83948,Torres,RS,29°21'S,49°50'W,4.66,-29.35,-49.833333,0.000000,1990-12-30,1990,12


In [692]:
df_result['Date'] = pd.to_datetime(df_result['Date'])
df_result['Year'] = df_result['Date'].dt.year
df_result['Month'] = df_result['Date'].dt.month
df_result['Date'] = df_result['Date'].astype(str)
df_result          

,Código,Nome,UF,Latitude,Longitude,Altitude (m),Latitude_Decimal,Longitude_Decimal,Value_Interpolated,Date,Year,Month
0,83010,Brasiléia,AC,11°01'S,68°44'W,260.00,-11.016667,-68.733333,4.278750,1961-01-01,1961,1
1,82704,Cruzeiro do Sul,AC,07°38'S,72°40'W,170.00,-7.633333,-72.666667,0.028541,1961-01-01,1961,1
2,82915,Rio Branco,AC,09°58'S,67°48'W,160.00,-9.966667,-67.800000,2.152689,1961-01-01,1961,1
3,82807,Tarauacá,AC,08°10'S,70°46'W,190.00,-8.166667,-70.766667,3.906242,1961-01-01,1961,1
4,83098,Coruripe,AL,10°09'S,36°10'W,56.13,-10.150000,-36.166667,0.000000,1961-01-01,1961,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,12°33'S,47°50'W,275.00,-12.550000,-47.833333,0.773416,1990-12-31,1990,12
4317054,82863,Pedro Afonso,TO,08°58'S,48°11'W,187.00,-8.966667,-48.183333,0.299561,1990-12-31,1990,12
4317055,83228,Peixe,TO,12°01'S,48°21'W,242.49,-12.016667,-48.350000,5.480674,1990-12-31,1990,12
4317056,83064,Porto Nacional,TO,10°43'S,48°25'W,239.20,-10.716667,-48.416667,8.465086,1990-12-31,1990,12


In [693]:
df_result.to_hdf(cn_path_prec + '/BRASIL_CN_INTERPOLATED_VALUES_1961_1990.h5', 'table_data', mode = 'w', append = False, complevel = 9, encoding="cp860")

In [694]:
df_interpolation = pd.read_hdf(cn_path_prec + '/BRASIL_CN_INTERPOLATED_VALUES_1961_1990.h5', 'table_data')
df_interpolation

,Código,Nome,UF,Latitude,Longitude,Altitude (m),Latitude_Decimal,Longitude_Decimal,Value_Interpolated,Date,Year,Month
0,83010,Brasiléia,AC,11°01'S,68°44'W,260.00,-11.016667,-68.733333,4.278750,1961-01-01,1961,1
1,82704,Cruzeiro do Sul,AC,07°38'S,72°40'W,170.00,-7.633333,-72.666667,0.028541,1961-01-01,1961,1
2,82915,Rio Branco,AC,09°58'S,67°48'W,160.00,-9.966667,-67.800000,2.152689,1961-01-01,1961,1
3,82807,Tarauacá,AC,08°10'S,70°46'W,190.00,-8.166667,-70.766667,3.906242,1961-01-01,1961,1
4,83098,Coruripe,AL,10°09'S,36°10'W,56.13,-10.150000,-36.166667,0.000000,1961-01-01,1961,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,12°33'S,47°50'W,275.00,-12.550000,-47.833333,0.773416,1990-12-31,1990,12
4317054,82863,Pedro Afonso,TO,08°58'S,48°11'W,187.00,-8.966667,-48.183333,0.299561,1990-12-31,1990,12
4317055,83228,Peixe,TO,12°01'S,48°21'W,242.49,-12.016667,-48.350000,5.480674,1990-12-31,1990,12
4317056,83064,Porto Nacional,TO,10°43'S,48°25'W,239.20,-10.716667,-48.416667,8.465086,1990-12-31,1990,12
